In [239]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy
import random
import scipy.sparse as sp
from time import time

# directory = '../MF/ml-1m'
# ratings = []
# with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
#     for l in f:
#         user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
#         ratings.append({
#             'user_id': user_id,
#             'movie_id': movie_id,
#             'rating': rating,
#             'timestamp': timestamp,
#             })
# ratings = pd.DataFrame(ratings)

# ratings = ratings.drop(['timestamp'],axis=1)
# print(ratings)
# # 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
# df = ratings.sample(frac=1).reset_index(drop=True)
# filter_user = deepcopy(df)
# counts = filter_user['user_id'].value_counts()
# filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

# filtered_df = deepcopy(filter_user)
# counts = filtered_df['movie_id'].value_counts()
# filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
# filtered_df = filtered_df.reset_index(drop=True)
# print(filtered_df)


class generate_graph(object):
    def __init__(self, path):
        self.path = path  # '../NGCF-PyTorch/Data/ml-1m'
        train_file = path + '/train.txt'
        
        self.neighbor_dict = {}
        self.user, self.item = [], []
        
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    self.neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
                    self.user.append(int(l[0]))
        self.user = user[:1000]
        
    def jaccard_index(self, u_i, u_j, neighbor_dict):
        u_i_neighbor = self.neighbor_dict[u_i]
        u_j_neighbor = self.neighbor_dict[u_j]
        return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

    # 새로운 graph의 node j 에다가 기존 graph의 어떤 node의 neighborhood를 복사할지 zeta에 담기
    def node_copying(self):
        t1 = time()
        zeta = []
        
        for u_j in self.user:
            nor = 0
            zeta_distribution = []
            for u_i in self.user:
                nor+=jaccard_index(u_j, u_i, self.neighbor_dict)
            for u_m in self.user:
                zeta_distribution.append(jaccard_index(u_j, u_m, self.neighbor_dict) / nor)
            zeta.append(random.choices(self.user, weights=zeta_distribution)[0])
        print('total node copying time cost : ', time() - t1)
        return zeta
    
    def generate_graph(self, epsilon, iteration):
        t2 = time()
        self.epsilon = epsilon
        self.iteration = iteration
        
        generated_node = []
        
        with open(self.path+'/sampled_graph/sampled_graph_'+str(iteration+1), 'w') as f:
            for i in self.user:
                if random.uniform(0,1) < 1-self.epsilon:  # 1-epsilon의 확률로 원래 neighbor 넣기
                    generated_node.append(i)
                else:                                # epsilon의 확률로 zeta에 있는 node의 neighbor로 copy해서 넣기
                    generated_node.append(zeta[i])
                    
                # 만들어지는 새로운 graph를 txt 파일로 저장
                f.write(str(i))
                f.write(' ')
                for j in self.neighbor_dict[generated_node[i]][:-1]:
                    f.write(str(j))
                    f.write(' ')
                f.write(str(self.neighbor_dict[generated_node[i]][-1]))
                f.write('\n')
        print(iteration+1,'-th Graph sampled time cost : ', time() - t2)

In [240]:
t = generate_graph('ml-1m')
zeta = t.node_copying()

zeta
# zeta = node_copying(user)

total node copying time cost :  33.542524337768555


[185,
 961,
 175,
 274,
 797,
 92,
 393,
 595,
 576,
 165,
 87,
 978,
 776,
 330,
 504,
 368,
 577,
 259,
 766,
 561,
 343,
 170,
 809,
 738,
 779,
 678,
 836,
 27,
 273,
 616,
 397,
 155,
 923,
 480,
 177,
 763,
 389,
 175,
 779,
 120,
 938,
 719,
 285,
 773,
 356,
 119,
 824,
 572,
 821,
 361,
 850,
 795,
 586,
 898,
 491,
 908,
 743,
 986,
 373,
 180,
 364,
 615,
 783,
 564,
 235,
 299,
 35,
 101,
 408,
 497,
 64,
 71,
 738,
 26,
 197,
 348,
 717,
 573,
 18,
 689,
 159,
 168,
 458,
 112,
 775,
 292,
 609,
 638,
 48,
 747,
 54,
 735,
 859,
 93,
 809,
 541,
 930,
 209,
 924,
 709,
 272,
 223,
 138,
 76,
 925,
 387,
 768,
 223,
 711,
 5,
 41,
 55,
 508,
 835,
 621,
 412,
 408,
 799,
 391,
 302,
 199,
 68,
 301,
 238,
 662,
 474,
 9,
 549,
 706,
 376,
 133,
 678,
 81,
 866,
 788,
 838,
 87,
 931,
 823,
 928,
 314,
 112,
 393,
 806,
 447,
 622,
 579,
 230,
 668,
 378,
 935,
 449,
 146,
 469,
 665,
 924,
 576,
 322,
 273,
 992,
 630,
 65,
 352,
 887,
 385,
 89,
 861,
 819,
 952,
 156,
 60

In [242]:
# zeta를 한번 만든 후 iteration마다 generate graph하는 방식

for epoch in range(3):
    t.generate_graph(epsilon = 0.01, iteration = epoch)

1 -th Graph sampled time cost :  0.028918027877807617
2 -th Graph sampled time cost :  0.024115562438964844
3 -th Graph sampled time cost :  0.02367234230041504


In [ ]:
# GAT
# 만들어지는 graph마다 embedding 초기화 
# embedding 학습하는 GNN 구조 짜기
# 학습된 embedding 가지고 x_hat (eq.11) 구하고 BPR-OPT를 maximization 시키는 방향으로 학습

In [202]:
# data로 sparse matrix 및 adjacency matrix 만들기

# TODO : node copying으로 만들어진 graph들에 대해서 adj matrix 만들고 npz 저장하는 과정 

class Data(object):
    def __init__(self, path):
        self.path =path
        
        train_file = path + '/train.txt'
        # path : ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(n_users, uid)
                    self.n_items = max(n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(n_users, n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


6040 3953
  (0, 720)	1.0
  (0, 1836)	1.0
  (0, 3186)	1.0
  (0, 2028)	1.0
  (0, 745)	1.0
  (0, 3114)	1.0
  (0, 595)	1.0
  (0, 1035)	1.0
  (0, 3105)	1.0
  (0, 914)	1.0
  (0, 783)	1.0
  (0, 1028)	1.0
  (0, 1545)	1.0
  (0, 527)	1.0
  (0, 1)	1.0
  (0, 2398)	1.0
  (0, 1907)	1.0
  (0, 1207)	1.0
  (0, 2340)	1.0
  (0, 1962)	1.0
  (0, 594)	1.0
  (0, 1029)	1.0
  (0, 1197)	1.0
  (0, 2687)	1.0
  (0, 2018)	1.0
  :	:
  (6039, 30)	1.0
  (6039, 1641)	1.0
  (6039, 1219)	1.0
  (6039, 2349)	1.0
  (6039, 1449)	1.0
  (6039, 592)	1.0
  (6039, 1127)	1.0
  (6039, 1150)	1.0
  (6039, 17)	1.0
  (6039, 3224)	1.0
  (6039, 3168)	1.0
  (6039, 265)	1.0
  (6039, 2022)	1.0
  (6039, 296)	1.0
  (6039, 2872)	1.0
  (6039, 1212)	1.0
  (6039, 1419)	1.0
  (6039, 2692)	1.0
  (6039, 2407)	1.0
  (6039, 1249)	1.0
  (6039, 2391)	1.0
  (6039, 25)	1.0
  (6039, 3068)	1.0
  (6039, 910)	1.0
  (6039, 1193)	1.0


In [ ]:
# data로 sparse matrix 및 adjacency matrix 만들기

# TODO : node copying으로 만들어진 graph들에 대해서 adj matrix 만들고 npz 저장하는 과정 

class sampled_graph_to_matrix(object):
    def __init__(self, path, iteration):
        self.path =path
        self.iteration = iteration
        
        sampled_graph = path + '/sampled_graph_' + iteration
        # path : ../NGCF-PyTorch/Data/ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(n_users, uid)
                    self.n_items = max(n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(n_users, n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [137]:
class AttenConv(nn.Module):
    def __init__(self):
        super(AttenConv, self).__init__()
    
    def forward(self):
        pass

In [193]:
a = np.matrix([[0,1,2],[1,1,1],[0,0,1]])
adj = np.matrix([[0,0,1],[1,0,0],[1,1,0]])
e = np.matrix([[1,2,3],[4,5,6],[7,8,9]])

print(a,'\n')
print(adj,'\n')
print(e,'\n')

a_jk = np.multiply(a,adj)
print(a_jk, '\n')

np.matmul(a_jk, e)
    
    
    

[[0 1 2]
 [1 1 1]
 [0 0 1]] 

[[0 0 1]
 [1 0 0]
 [1 1 0]] 

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[0 0 2]
 [1 0 0]
 [0 0 0]] 



matrix([[14, 16, 18],
        [ 1,  2,  3],
        [ 0,  0,  0]])